First we install boto3 for later access to s3

In [1]:
!pip install boto3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 76.2 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 13.0 MB/s eta 0:00:00


And load the secrets from secrets.env file

In [2]:
from dotenv import dotenv_values

config = dotenv_values("secrets.env")

Here we put our write token from ion, and select a name and description for the initial upload

In [3]:
import requests
import json
headers={'Authorization': f"""Bearer {config['ion_token']}""",
        'Content-Type': 'application/json'}

payload = json.dumps({
  "name": "test",
  "type": "3DTILES",
  "description":"",
  "options": {
    "sourceType": "POINT_CLOUD"
  }
})

url='https://api.cesium.com/v1/assets'
response = requests.request("POST", url, headers=headers, data=payload)

print(response.text)

{"code":"Forbidden","message":"Request exceeds quote"}


We then extract the upload location

In [ ]:
print(response.json()["uploadLocation"])
uploadLocation=response.json()["uploadLocation"]

And prepare the s3 client

In [ ]:
import boto3

client = boto3.client(
    's3',
    aws_access_key_id=uploadLocation['accessKey'],
    aws_secret_access_key=uploadLocation['secretAccessKey'],
    aws_session_token=uploadLocation['sessionToken']
)


Please provide your file's name as file_name below for upload

In [ ]:
file_name="NUE_20170721_M1_RGB_group1_densified_point_cloud.laz"
file_name="GegedzerickRd_test_odm_georeferenced_model.laz"

Check the file size and available storage.

In [ ]:
import os
import requests

file_size=os.path.getsize(file_name)
print("file size is " + str(file_size) + " bytes")

me_response = requests.request("GET", "https://api.cesium.com/v1/me", headers=headers)
available_storage=me_response.json()['storage']['available']
print("available storage is " + str(available_storage) + " bytes")

if available_storage>=file_size:
    print("There is enough space")
else:
    print("There is not enough space")

And start the upload process

In [ ]:
import logging

bucket=uploadLocation['bucket']
object_name=uploadLocation['prefix']+file_name
try:
    upload_response = client.upload_file(file_name, bucket, object_name)
except ClientError as e:
    logging.error(e)

Or provide its asdc url

In [ ]:
file_url="https://asdc.cloud.edu.au/api/projects/780/tasks/68dc4f32-d107-4af6-96c6-c33375b65f23/download/georeferenced_model.laz"
# r = requests.get(url, allow_redirects=True)
# open('file.laz', 'wb').write(r.content)

import asdc
asdc.download(file_url,"file.laz",overwrite=True)
file_name="file.laz"
bucket=uploadLocation['bucket']
object_name=uploadLocation['prefix']+file_name
try:
    upload_response = client.upload_file(file_name, bucket, object_name)
except ClientError as e:
    logging.error(e)

Once the upload finishes we send the onComplete request to start the tiling

In [ ]:
onCompleteResponse = requests.request("POST", response.json()["onComplete"]["url"], headers=headers, data={})


And we track its progress

In [ ]:
from threading import Timer

assetID=response.json()["assetMetadata"]["id"]
print(assetID)

def waitUntilReady():
    url='https://api.cesium.com/v1/assets/'+str(assetID)
    assetMetadata = requests.request("GET", url, headers=headers, data={})
    # print(assetMetadata.text)
    status=assetMetadata.json()["status"]
    # print(status)
    
    if status=='COMPLETE':
        print("asset tiled successfully")
    elif status=='DATA_ERROR':
        print('ion detected a problem with the uploaded data.')
    elif status=='ERROR':
        print("An unknown tiling error occurred")
    elif status=='AWAITING_FILES':
        print("Awaiting file upload")
    else:
        if status=="NOT_STARTED":
            print("tiling pipeline initialising.")
        elif status=="IN_PROGRESS":
            print(f"""asset is {assetMetadata.json()["percentComplete"]}% complete""")
            
        t=Timer(10,waitUntilReady)
        t.start()
    

    
waitUntilReady()

Finally, we send the export request to export the tiles to our s3 account

In [ ]:
assetID = 2330332
url=f"""https://api.cesium.com/v1/assets/{assetID}/exports"""
# assetID = 2330332
payload=json.dumps({
    "type":"S3",
    "bucket":"appf-anu",
    "prefix":f"""/Cesium/ion_exports/{assetID}""",
    "accessKeyId":config['accessKeyId'],
    "secretAccessKey":config["secretAccessKey"],
    "sessionToken":""
})
exportReq = requests.request("POST", url, headers=headers, data=payload)
print(exportReq.text)

We can then track the status of this export request

In [ ]:
from threading import Timer #this is needed if we are skipping the Cesiuim import step and just uploading an existing asset to S3

export_id=exportReq.json()["id"]
def wait_until_export_complete():
    url='https://api.cesium.com/v1/assets/'+str(assetID)+'/exports/'+str(export_id)
    export_status_req = requests.request("GET", url, headers=headers, data={})
    # print(export_status_req.text)
    export_status=export_status_req.json()["status"]
    print(export_status)
    
    if export_status!="COMPLETE":
        t=Timer(2,wait_until_export_complete)
        t.start()
        
wait_until_export_complete()

To view the output in cesium, we first make the tileset public on amazon. Please note, we need to wait for the export above to finish first

In [11]:
import boto3

s3 = boto3.resource('s3',
    aws_access_key_id=config['accessKeyId'],
    aws_secret_access_key=config["secretAccessKey"])

bucket = s3.Bucket('appf-anu')

for obj in bucket.objects.filter(Prefix=f"""Cesium/ion_exports/{assetID}"""):
    # print(obj)
    object_acl = s3.ObjectAcl(obj.bucket_name,obj.key)
    object_acl.put(ACL='public-read')
    

We then create the index file for the cesium interface

In [12]:
index={
    "assets":[
        {
            "id": 1,
            "name": "Ion",
            "status": "active",
            "categoryID": 1,
            "data": [
                1
            ]
        }
    ],
    "datasets":[
        {
            "name": assetID,
            "type": "PointCloud",
            "url": f"""https://appf-anu.s3.ap-southeast-2.amazonaws.com/Cesium/ion_exports/{assetID}/tileset.json""",
            "id": 1
          }
    ],
    "categories": [
        {
            "id": 1,
            "name": "Export"
        }
    ]
}

import json
with open(f"""{assetID}.json""", 'w') as fp:
    json.dump(index, fp)

And upload the index file to s3

In [13]:
s3_client = boto3.client(
    's3',
    aws_access_key_id=config['accessKeyId'],
    aws_secret_access_key=config["secretAccessKey"]
)
file_name=f"""{assetID}.json"""
bucket="appf-anu"
object_name=f"""Cesium/ion_exports/{assetID}.json"""
try:
    upload_response = s3_client.upload_file(file_name, bucket, object_name,ExtraArgs={'ACL':'public-read'})
except ClientError as e:
    logging.error(e)

The tileset can then be seen at the link below

In [ ]:
index_file_path=f"""https://appf-anu.s3.ap-southeast-2.amazonaws.com/Cesium/ion_exports/{assetID}.json"""
print(f"""https://cesium.asdc.cloud.edu.au/cesium/Apps/ASDC/?index={index_file_path}""")

We then delete the asset from ion

In [15]:
url='https://api.cesium.com/v1/assets/'+str(assetID)
deleteReq = requests.request("DELETE", url, headers=headers, data=payload)

We first create a project

In [ ]:
import asdc
data={"description": "",
"name": f"""{assetID}""",
"tags": []}
r = asdc.call_api("https://asdc.cloud.edu.au/api/projects/",data=data)

In [ ]:
print(r.text)
print(r.json()["id"])
project_id=r.json()["id"]

We then create a task in this project

In [ ]:
options = {
    "auto-boundary": True,
    "dsm": True
}
# convert into list with "name" / "value" dictionaries, suitable for ODM
options_list = [{"name": k, "value": v} for k, v in options.items()]
data = {
    "partial": True,
    "name": f"""ion export - {assetID}""",
    "options": options_list
}

url = f"/projects/{project_id}/tasks/"
res = asdc.call_api(url, data=data)

In [ ]:
{
"type": "FeatureCollection",
"name": "SELECT",
"crs": { "type": "name", "properties": { "name": "urn:ogc:def:crs:OGC:1.3:CRS84" } },
"features": [
{ "type": "Feature", "properties": { }, "geometry": { "type": "Polygon", "coordinates": [ [ [ 149.094708174653675, -35.278852751857457 ], [ 149.094712211572357, -35.260468935814821 ], [ 149.130447249083545, -35.260468935814821 ], [ 149.13045128600217, -35.278852751857457 ], [ 149.094708174653675, -35.278852751857457 ] ] ] } }
]
}


In [ ]:
print(res.text)
task_id=res.json()["id"]
print(task_id)

We upload the index file to the task as an asset

In [ ]:
r = asdc.upload_asset(f"""{assetID}.json""", dest=f"""{assetID}.json""", task=task_id)


We make the task public to access the asset.

In [ ]:
import asdc
# await asdc.auth.connect()
token = asdc.auth.access_token
url = 'https://asdc.cloud.edu.au'
assert(token)

In [ ]:
from odk2odm import odm_requests
res =odm_requests.patch_task(url, token, project_id, task_id, data={"public":"true"})

We can then view the dataset in cesium using the link below

In [ ]:
index_path=f"""https://asdc.cloud.edu.au/api/projects/{project_id}/tasks/{task_id}/assets/{assetID}.json"""
print(f"""https://cesium.asdc.cloud.edu.au/cesium/Apps/ASDC/?index={index_path}""")